# Using Ivy ResNet18

Use the Ivy `ResNet18` model for image classification.

If you already have `Ivy` and `Ivy Models` installed you can skip this cell, but if you are using Colab, you will have to install `Ivy` and `Ivy Models` manually. You can do so by running the cell below ⬇️

Keep in mind that for the package to be available, you will have to click on **"Runtime > Restart Runtime"** 😄

In [ ]:
!git clone https://github.com/unifyai/ivy.git
!cd ivy && git checkout 5cbfea51b617c5fd6e17b6a508f1d67134775670 && python3 -m pip install -e .

!git clone https://github.com/unifyai/models.git
!cd models && python3 -m pip install -e .

! python3 -m pip install torchvision

To use the compiler and the transpiler now you will need an API Key. If you already have one, you should replace the string in the next cell.

In [2]:
API_KEY = "PASTE_YOUR_KEY_HERE"

In [3]:
!mkdir -p .ivy
!echo -n $API_KEY > .ivy/key.pem

## Imports

In [ ]:
import ivy
import torch

## Data Preparation

### Prepare the set of labels

To show the predicted category, we download the labels associated with the pretrained weights.

In [ ]:
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

The labels are then loaded into a Python list.

In [11]:
# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

### Load the image example 🖼️

In [12]:
# import numpy to load the image example .npy
import numpy as np 
img_path = "models/ivy_models_tests/img_resnet.npy"
img = np.load(img_path)

## Model Inference

### Initializing Native Torch ResNet18

In [5]:
from torchvision.models import resnet18, ResNet18_Weights
torch_resnet = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

### Initializing Ivy ResNet18 with Pretrained Weights through Ivy Container ⬇️

The pickled weights are loaded into `Ivy ResNet` as `ivy.Container`.

In [6]:
# equivalent to ResNet18_Weights.IMAGENET1K_V1
pretrained_weights_path = "models/ivy_models/resnet/pretrained_weights/resnet_18.pickled" 

In [7]:
weights = ivy.Container.cont_from_disk_as_pickled(pretrained_weights_path) 

The model is then initialized with the Pretrained Weights 🔗.

In [ ]:
# load the resnet18 model from ivy_models
from ivy_models.resnet import resnet_18 
ivy_resnet = resnet_18(v=weights) 

Compile the forward pass for efficiency. \
PS: Comment out the code below if you don't have an API key.

In [22]:
ivy.set_backend('torch')
ivy_resnet.compile(args=(img,))

### Use the model to classify your images 🚀

For comparison, both results from `Torch ResNet18` and `Ivy ResNet18` are shown below.

1. Torch ResNet18

In [23]:
torch_output = torch.softmax(torch_resnet(torch.from_numpy(img).reshape(1, 3, 224, 224)), dim=1)
torch_classes = torch.argsort(torch_output[0])[-4:]
torch_logits = torch.take(torch_output[0], torch_classes)

print("Indices of the top 4 classes are:", torch_classes)
print("Logits of the top 4 classes are:", torch_logits)
print("Categories of the top 4 classes are:", [categories[i] for i in torch_classes])

Indices of the top 4 classes are: tensor([412, 731, 600, 463])
Logits of the top 4 classes are: tensor([0.0055, 0.0064, 0.0065, 0.0082], grad_fn=<TakeBackward0>)
Categories of the top 4 classes are: ['ashcan', 'plunger', 'hook', 'bucket']


2. Ivy ResNet18

In [24]:
output = ivy.softmax(ivy_resnet(ivy.asarray(img))["w"])  # pass the image to the model
classes = ivy.argsort(output[0])[-4:]  # get the top 4 classes
logits = ivy.gather(output[0], classes)  # get the logits

print("Indices of the top 4 classes are:", classes)
print("Logits of the top 4 classes are:", logits)
print("Categories of the top 4 classes are:", [categories[i] for i in classes.to_list()])

Indices of the top 4 classes are: tensor([412, 731, 600, 463])
Logits of the top 4 classes are: [0.00549888 0.005922   0.00626743 0.00846343]
Categories of the top 4 classes are: ['ashcan', 'plunger', 'hook', 'bucket']
